# ①ライブラリのインポート

In [1]:
# データハンドリング
import numpy as np
import pandas as pd
# 正規化
from sklearn.preprocessing import MinMaxScaler
# データ分割
from sklearn.model_selection import train_test_split
# ニューラルネット
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
# 混同行列可視化用
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
# 正解率の算出
from sklearn.metrics import accuracy_score

# ②データの読み込みと分割

前章で加工済みデータをCSVにしていたので、それを読み込み、ニューラルネットの入力データに加工する

In [2]:
# 読み込み
data=pd.read_csv('data_complete.csv')

#説明変数と目的変数に分割
X=data[
    [
        'Pclass', 'Age', 'SibSp', 'Parch', 
        'Fare', 'Sex_female','Sex_male', 
        'Embarked_C', 'Embarked_Q', 'Embarked_S'
    ]
]
y=data['Survived']

# 訓練データとテストデータに分割
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234)

# 正規化
scaler_x = MinMaxScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test) 

# ③ニューラルネット（回帰）

回帰を試す理由としては、某企業の内部調査で、確率推論などの特定のタスクでは、
分類系のアルゴリズムより、回帰系アルゴリズムのほうが良い結果を示している。
また、今回の問題でも、「生きるか死ぬか」という分類問題であるが、
生きる確率0%/100%の回帰問題とも考えることができるため、回帰を試してみる。

In [3]:
"""基本モデルの作成：
"""
clf=MLPRegressor(
    random_state=1234
)

In [4]:
"""モデルの最適化
"""
clf.fit(X_train,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1234, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [5]:
"""正解率の算出
"""
y_train_predict=clf.predict(X_train)

y_train_predict_01=[1 if i>0.5 else 0 for i in y_train_predict]

print("正解率:",round(accuracy_score(y_train,y_train_predict_01),3))

正解率: 0.816


# ④差分の予測

③で求めた精度は、旋回の分類の精度に比べ、低くなりましたが、回帰系のアルゴリズムで、精度向上の一つとして、用いられる「差分の予測」を試してみる。

→これは、あまり一般的ではないが、「最終的な」精度向上段階で用いられる手法である。

→前段階のニューラルネットで吸収できなかった特徴を、後段階のニューラルネットで吸収しようという発想の試みである。

In [6]:
"""差分推論用のモデル
"""
clf_diff=MLPRegressor(
    random_state=1234
)

In [7]:
"""目的変数となる予測値との差分を求める
"""
train_diff=y_train-y_train_predict

In [8]:
"""最適化
"""
clf_diff.fit(X_train,train_diff)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1234, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [9]:
"""差分の予測値の算出
"""
train_diff_predict=clf_diff.predict(X_train)

In [10]:
"""最終的な予測値=予測値+差分の予測値
"""
train_predict_union=y_train_predict+train_diff_predict

In [11]:
"""正解率の算出
"""
train_predict_union_01=[1 if i>0.5 else 0 for i in train_predict_union]

print("正解率:",round(accuracy_score(y_train,train_predict_union_01),3))

正解率: 0.809


#### 差分の予測は、今回の問題ではあまり意味をなしていないことがわかる。

#### しかし、これは、この手法が無意味ということではなく、「今回の問題」では、成果を上げられなかったというだけなので、このような手法もあるのだなということは頭に残しておいてください。


もう少し学習を進めると、違う構造のニューラルネットをいくつも組み合わせて精度を上げるということは、往々にして出会うと思いますが、通常のデータ分析業務で使用することはまれなので、慣れないうちは、あまりトリッキーなことをしないほうがいいかもしれません。